In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.
# Python native
import pickle
from statistics import median, mean
import pandas as pd
# Data handling
# Object centric process mining
# import ocpa.algo.evaluation.precision_and_fitness.utils as evaluation_utils # COMMENTED OUT BY TIM
# import ocpa.algo.evaluation.precision_and_fitness.evaluator as precision_fitness_evaluator # COMMENTED OUT BY TIM
import ocpa.objects.log.importer.csv.factory as csv_import_factory
import ocpa.algo.predictive_monitoring.factory as feature_factory
from ocpa.objects.log.ocel import OCEL

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# # Custom GNN tools
# from gnn_utils import (
#     generate_graph_dataset,
#     # get_ordered_event_list,
#     # visualize_graph,
#     # show_remaining_times,
#     # visualize_instance,
#     # GraphDataLoader,
#     # GCN,
#     # evaluate_gnn,
# )

# PyG
import torch
from torch_geometric.loader import DataLoader
from replicating.ocpa_PyG_integration.EventGraphDataset import EventGraphDataset
from replicating.ocpa_PyG_integration.EventSubGraphDataset import EventSubGraphDataset
from replicating.model import GCN, GAT

# Global variables
from replicating.experiment_config import STORAGE_PATH, FEATURE_STORAGE_FILE, RANDOM_SEED, TARGET_LABEL


filename = "data/adams/example_logs/mdl/BPI2017-Final.csv"
object_types = ["application", "offer"]
parameters = {
    "obj_names": object_types,
    "val_names": [],
    "act_name": "event_activity",
    "time_name": "event_timestamp",
    "sep": ",",
}
file_path_object_attribute_table = None

Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.2.0
Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.2.0


In [2]:
ds_val = EventSubGraphDataset(
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    validation=True,
    verbosity=51
)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = GCN(24, {"num_hidden_features": 24})
loss_fn = torch.nn.L1Loss()
model = model.to(device)
val_loader = DataLoader(ds_val, batch_size=64, shuffle=True)
running_vloss = 0.0

In [4]:

for i, vdata in enumerate(val_loader):
            vdata.to(device)
            vinputs, vadjacency_matrix, vlabels = (
                vdata.x.float(),
                vdata.edge_index,
                vdata.y.float(),
            )
            voutputs = model(vinputs, vadjacency_matrix)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([156])) that is different to the input size (torch.Size([156, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [5]:
ds_train = EventSubGraphDataset(
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    train=True,
    verbosity=51
)
ds_val = EventSubGraphDataset(
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    validation=True,
    verbosity=51
)
ds_test = EventSubGraphDataset(
    root=STORAGE_PATH,
    filename=FEATURE_STORAGE_FILE,
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    test=True,
    verbosity=51
)

Processing...
15754it [01:22, 190.43it/s]
Done!
Processing...
6302it [00:31, 200.10it/s]
Done!
Processing...
9453it [00:49, 191.69it/s]
Done!


In [8]:
ds_train.len()

150012

In [ ]:
# Importing OCEL
ocel = csv_import_factory.apply(
    filename, csv_import_factory.TO_OCEL, parameters, file_path_object_attribute_table
)

In [ ]:
with open(f"debug/self._mapping-pass.pkl", "rb") as file:
    mapping_pass = pickle.load(file)
with open("debug/attribute-pass.pkl", "rb") as file:
    attribute_pass = pickle.load(file)
with open("debug/e_id-pass.pkl", "rb") as file:
    e_id_pass = pickle.load(file)
with open("debug/self._mapping[attribute][e_id]-pass.pkl", "rb") as file:
    value_pass = pickle.load(file)

with open(f"debug/self._mapping-fail.pkl", "rb") as file:
    mapping_fail = pickle.load(file)
with open("debug/attribute-fail.pkl", "rb") as file:
    attribute_fail = pickle.load(file)
with open("debug/e_id-fail.pkl", "rb") as file:
    e_id_fail = pickle.load(file)

with open("debug/ocel.log", "rb") as file:
    ocel_log = pickle.load(file)

In [ ]:
map = pd.DataFrame(ocel.log._mapping)
map.head()

In [ ]:
ocel.parameters

In [ ]:
mapping_pass = pd.DataFrame(mapping_pass)
print(attribute_pass)
print(e_id_pass)
print(value_pass)
mapping_pass.head()

In [ ]:
mapping_fail = pd.DataFrame(mapping_fail)
print(attribute_fail)
print(e_id_fail)
mapping_fail.head()

In [ ]:
try:
            self._mapping[attribute][e_id]
            with open("self._mapping-pass.pkl", "wb") as file:
                pickle.dump(self._mapping, file)
            with open("attribute-pass.pkl", "wb") as file:
                pickle.dump(attribute, file)
            with open("e_id-pass.pkl", "wb") as file:
                pickle.dump(e_id, file)
            with open("self._mapping[attribute][e_id]-pass.pkl", "wb") as file:
                pickle.dump(self._mapping[attribute][e_id], file)
        except:
            with open("self._mapping-fail.pkl", "wb") as file:
                pickle.dump(self._mapping, file)
            with open("attribute-fail.pkl", "wb") as file:
                pickle.dump(attribute, file)
            with open("e_id-fail.pkl", "wb") as file:
                pickle.dump(e_id, file)